In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import cv2
import os
import PIL
from PIL import Image
from unidecode import unidecode
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
df1 = pd.read_excel('wclassesFiltered.xlsx')


In [ ]:
train = df1.sample(frac=0.8,random_state=200)
test = df1.drop(train.index)

In [ ]:
im = cv2.imread(train.iloc[4, 0])
print(im.shape)
# print(train.iloc[0, 0])

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  r'C:\Users\biamc\Documents\Insper\Machine Learning\SArt',
  labels='inferred',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(256, 256),
  batch_size=32)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(27))

In [ ]:
model.summary()


In [ ]:
target_size=(256,256) # set the size of the images
color_mode='rgb' # set the type of image
class_mode= 'categorical' # set the class mode
batch_size=32  # set the batch size
subset='training' # set to 'training', or 'valiatiom' or leave as None

train_gen = ImageDataGenerator()
test_gen = ImageDataGenerator()
train_imgs= train_gen.flow_from_dataframe(dataframe=train, 
          x_col='file',
          y_col=['style'],
          target_size=target_size,color_mode=color_mode,
          class_mode='raw', batch_size=batch_size,shuffle=True, seed=123,
          subset='training')
test_imgs= test_gen.flow_from_dataframe(dataframe=test, 
          x_col='file',
          y_col=['style'],
          target_size=target_size,color_mode=color_mode,
          class_mode='raw', batch_size=batch_size,shuffle=True, seed=123,
          subset='training')

In [ ]:
import scipy
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
train_labels = train["style"].to_numpy()
test_labels = test["style"].to_numpy()
print(type(train_labels))

history = model.fit(train_imgs, epochs=10, 
                    validation_data=(test_imgs))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_imgs, verbose=2)